In [ ]:
import argparse
import pandas as pd
import numpy as np
import math
import h5py
from sklearn.model_selection import train_test_split
import joblib
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import sys
import gc

# import setGPU
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Concatenate, Dropout, Layer
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras import backend as K
tf.keras.mixed_precision.set_global_policy('mixed_float16')

from datetime import datetime
from tensorboard import program
import os
import pathlib
import matplotlib as mpl
import matplotlib.pyplot as plt
try:
    import mplhep as hep
    hep.style.use(hep.style.ROOT)
    print("Using MPL HEP for ROOT style formating")
except:
    print("Instal MPL HEP for style formating")
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["#DB4437", "#4285F4", "#F4B400", "#0F9D58", "purple", "goldenrod", "peru", "coral","turquoise",'gray','navy','m','darkgreen','fuchsia','steelblue']) 
#from autoencoder_classes import AE,VAE

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from neptunecontrib.monitoring.keras import NeptuneMonitor
from losses import mse_split_loss, radius, kl_loss
from functions import make_mse_loss_numpy
from sklearn.metrics import roc_curve, auc
from functions import make_mse_loss


from data_preprocessing import prepare_data
#from model import build_AE, build_VAE, Sampling
from model import Sampling


def return_total_loss(loss, bsm_t, bsm_pred):
    total_loss = loss(bsm_t, bsm_pred.astype(np.float32))
    return total_loss

In [ ]:
####configuration####
global input_qcd, input_bsm, events, load_pickle, input_pickle, output_pfile, \
        output_model_h5, output_model_json, output_history, output_result, \
        model_type, n_epochs

input_qcd="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-h5-extended-v2/QCD_preprocessed.h5"
input_bsm="/eos/uscms/store/group/lpctrig/jngadiub/L1TNtupleRun3-h5-extended-v2/BSM_preprocessed.h5"
events = 1000000
load_pickle=False
input_pickle="data.pickle"
output_pfile="data.pickle"
output_model_h5='model.h5'
output_model_json='model.json'
output_history='history.h5'
output_result='results.h5'
model_type='VAE'
#batch_size= 1024
n_epochs = 150

In [ ]:
def main(hyperparameters):
    
    latent_dim = hyperparameters[:,0]
    outer_layer_width = hyperparameters[:,1]
    inner_layer_width = hyperparameters[:,2]
    beta = hyperparameters[:,3]
    batch_size = int(hyperparameters[:,4])
    
    class VAE(Model):
        def __init__(self, encoder, decoder, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
            self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
            self.total_val_loss_tracker = keras.metrics.Mean(name="total_val_loss")
            self.reconstruction_val_loss_tracker = keras.metrics.Mean(name="reconstruction_val_loss")
            self.kl_val_loss_tracker = keras.metrics.Mean(name="kl_val_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
                self.total_val_loss_tracker,
                self.reconstruction_val_loss_tracker,
                self.kl_val_loss_tracker
            ]

        def train_step(self, data):
            print('Beta is ', beta)
            data_in, target = data
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data_in, training=True)
                reconstruction = self.decoder(z, training=True)
            
                reconstruction_loss = make_mse_loss(target, reconstruction) #one value
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                kl_loss = tf.reduce_mean(kl_loss, axis=-1)
                kl_loss = tf.cast(kl_loss, tf.float32)
                total_loss = (1-beta)*reconstruction_loss + beta*kl_loss
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state((1-beta)*reconstruction_loss)
            self.kl_loss_tracker.update_state(beta*kl_loss)
        
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }

        def test_step(self, data):
            #validation
            data_in, target = data
            z_mean, z_log_var, z = self.encoder(data_in)
            reconstruction = self.decoder(z)
        
            reconstruction_loss = make_mse_loss(target, reconstruction) 
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss, axis=1)
            kl_loss = tf.cast(kl_loss, tf.float32)
            total_loss = (1-beta)*reconstruction_loss + beta*kl_loss
            self.total_val_loss_tracker.update_state(total_loss)
            self.reconstruction_val_loss_tracker.update_state((1-beta)*reconstruction_loss)
            self.kl_val_loss_tracker.update_state(beta*kl_loss)

            return {
                "loss": self.total_val_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_val_loss_tracker.result(),
                "kl_loss": self.kl_val_loss_tracker.result()
            }
    
        def save(self, path):
            pathlib.Path(path).mkdir(parents=True, exist_ok=True)
            print('saving model to {}'.format(path))
            self.encoder.save(os.path.join(path, 'encoder.h5'))
            self.decoder.save(os.path.join(path,'decoder.h5'))

        @classmethod
        def load(cls, path, custom_objects={}):
            ''' loading only for inference -> passing compile=False '''
            encoder = tf.keras.models.load_model(os.path.join(path,'encoder.h5'), custom_objects=custom_objects, compile=False)
            decoder = tf.keras.models.load_model(os.path.join(path,'decoder.h5'), custom_objects=custom_objects, compile=False)
            return encoder, decoder
    
        def build_AE(input_shape,latent_dim, outer_layer_width, inner_layer_width):
            inputArray = Input(shape=(input_shape))
            x = BatchNormalization()(inputArray)
            x = Dense(outer_layer_width, kernel_initializer=tf.keras.initializers.HeUniform())(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.3)(x)
            x = Dense(inner_layer_width, kernel_initializer=tf.keras.initializers.HeUniform())(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.3)(x)
            encoder = Dense(latent_dim, kernel_initializer=tf.keras.initializers.HeUniform())(x)
            # x = BatchNormalization()(x)
            # encoder = LeakyReLU(alpha=0.3)(x)
            #decoder
            x = Dense(inner_layer_width, kernel_initializer=tf.keras.initializers.HeUniform())(encoder)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.3)(x)
            x = Dense(outer_layer_width, kernel_initializer=tf.keras.initializers.HeUniform())(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.3)(x)
            decoder = Dense(input_shape, kernel_initializer=tf.keras.initializers.HeUniform())(x)

            #create autoencoder
            autoencoder = Model(inputs = inputArray, outputs=decoder)
            autoencoder.summary()
            # ae = AE(autoencoder)
            # ae.compile(optimizer=keras.optimizers.Adam(lr=0.00001))

            return autoencoder
    
    def build_VAE(input_shape, latent_dim, outer_layer_width, inner_layer_width):
    
        #encoder
        inputArray = Input(shape=(input_shape))
        x = BatchNormalization()(inputArray)
        x = Dense(outer_layer_width, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.3)(x)
        x = Dense(inner_layer_width, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.3)(x)
        mu = Dense(latent_dim, name = 'latent_mu', kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        logvar = Dense(latent_dim, name = 'latent_logvar', kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)

        # Use reparameterization trick to ensure correct gradient
        z = Sampling()([mu, logvar])

        # Create encoder
        encoder = Model(inputArray, [mu, logvar, z], name='encoder')
        encoder.summary()

        #decoder
        d_input = Input(shape=(int(latent_dim),), name='decoder_input')
        x = Dense(inner_layer_width, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(d_input)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.3)(x)
        x = Dense(outer_layer_width, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.3)(x)
        dec = Dense(input_shape, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)

        # Create decoder 
        decoder = Model(d_input, dec, name='decoder')
        decoder.summary()
    
        # vae = VAE(encoder, decoder)
        # vae.compile(optimizer=keras.optimizers.Adam())

        return encoder,decoder
    
    if(load_pickle):
        if(input_pickle==''):
            print('Please provide input pickle files')
        with open(input_pickle, 'rb') as f:
            X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler, bsm_labels = pickle.load(f)
            bsm_labels=['VectorZPrimeToQQ__M50',
                  'VectorZPrimeToQQ__M100',
                  'VectorZPrimeToQQ__M200',
                  'VBF_HToInvisible_M125',
                  'VBF_HToInvisible_M125_private',
                  'ZprimeToZH_MZprime1000',
                  'ZprimeToZH_MZprime800',
                  'ZprimeToZH_MZprime600',
                  'GluGluToHHTo4B',
                  'HTo2LongLivedTo4mu_1000',
                  'HTo2LongLivedTo4mu_125_12',
                  'HTo2LongLivedTo4mu_125_25',
                  'HTo2LongLivedTo4mu_125_50',
                  'VBFHToTauTau',
                  'VBF_HH']
    else:
        if(input_qcd==''or input_bsm==''):
            print('Please provide input H5 files')
        X_train_flatten, X_train_scaled, X_test_flatten, X_test_scaled, bsm_data, bsm_target, pt_scaler, bsm_labels = prepare_data(input_qcd, input_bsm, events, output_pfile,True)
        
    if(model_type=='AE'):
        autoencoder = build_AE(X_train_flatten.shape[-1], latent_dim, outer_layer_width, inner_layer_width)
        model = AE(autoencoder)
        model.compile(optimizer=keras.optimizers.Adam(lr=0.001))

        callbacks=[]
        callbacks.append(ReduceLROnPlateau(monitor='val_loss',  factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
        callbacks.append(TerminateOnNaN())
        callbacks.append(NeptuneMonitor())
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=10, restore_best_weights=True))

    elif(model_type=='VAE'):
        encoder, decoder = build_VAE(X_train_flatten.shape[-1], latent_dim, outer_layer_width, inner_layer_width)
        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam())

        callbacks=[]
        callbacks.append(ReduceLROnPlateau(monitor='val_loss',  factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=2, min_lr=1E-6))
        callbacks.append(TerminateOnNaN())
        callbacks.append(NeptuneMonitor())
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss',verbose=1, patience=10, restore_best_weights=True))

    history = model.fit(X_train_flatten, X_train_scaled,
                        epochs=n_epochs,
                        batch_size=batch_size,
                        validation_split=0.2,
                        callbacks=callbacks)

    del X_train_flatten, X_train_scaled


    gc.collect()
    
    if(output_model_h5!=''):
        if(model_type=='VAE'):
            model.save(os.path.join(os.getcwd(),output_model_h5.split('.')[0]))
        else:
            model_json = autoencoder.to_json()
            with open(output_model_json, 'w') as json_file:
                json_file.write(model_json)
            autoencoder.save_weights(output_model_h5)


    if(output_history!=''):
        with open(output_history, 'wb') as f:
            pickle.dump(history.history, f)
    
    #load model
    model_dir = output_model_h5.split('.')[0]
    if(model_type=='AE'):
        with open(model_dir+"/model.json", 'r') as jsonfile: config = jsonfile.read()
        ae = tf.keras.models.model_from_json(config)    
        ae.load_weights(model_dir+"/model.h5")
        ae.summary()
        model = AE(ae)
    elif(model_type=='VAE'):
        encoder, decoder = VAE.load(model_dir, custom_objects={'Sampling': Sampling})
        encoder.summary()
        decoder.summary()
        model = VAE(encoder, decoder)
    
    from end2end import get_results
    data_file = input_pickle
    outdir = output_model_h5.split('.')[0]
    if not load_pickle: data_file = output_pfile
    results = get_results(input_qcd,input_bsm,data_file,outdir,events,model_type,latent_dim)   
    
    for key in results.keys():
        results[key]['loss'] = results[key]['loss'][np.isfinite(results[key]['loss'])]
        results[key]['total_loss'] = results[key]['total_loss'][np.isfinite(results[key]['total_loss'])]
        results[key]['radius'] = results[key]['radius'][np.isfinite(results[key]['radius'])]

    signal_eff={}

    for key in results.keys():
        if key=='QCD': continue
        signal_eff[key]={}
        true_label = np.concatenate(( np.ones(results[key]['loss'].shape[0]), np.zeros(results['QCD']['loss'].shape[0]) ))
        pred_loss = np.concatenate(( results[key]['loss'], results['QCD']['loss'] ))
        fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
        signal_eff[key]['MSE_loss']=tpr_loss[fpr_loss<0.000125][-1]

        auc_loss = auc(fpr_loss, tpr_loss)

    if(model_type=='VAE'):
        #plt.figure(figsize=(10,10))
        for key in results.keys():
            if key=='QCD': continue

            true_label = np.concatenate(( np.ones(results[key]['total_loss'].shape[0]), np.zeros(results['QCD']['total_loss'].shape[0]) ))
            pred_loss = np.concatenate(( results[key]['total_loss'], results['QCD']['total_loss'] ))
            fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
            signal_eff[key]['KL_loss']=tpr_loss[fpr_loss<0.000125][-1]

            auc_loss = auc(fpr_loss, tpr_loss)
  
        for key in results.keys():
            if key=='QCD': continue

            true_label = np.concatenate(( np.ones(results[key]['radius'].shape[0]), np.zeros(results['QCD']['radius'].shape[0]) ))
            pred_loss = np.concatenate(( results[key]['radius'], results['QCD']['radius'] ))
            fpr_loss, tpr_loss, threshold_loss = roc_curve(true_label, pred_loss)
            signal_eff[key]['radius']=tpr_loss[fpr_loss<0.000125][-1]
        
        
            auc_loss = auc(fpr_loss, tpr_loss)
    
    signal_eff_pd = pd.DataFrame.from_dict(signal_eff).transpose()

    #return auc_loss
    return -(tpr_loss[fpr_loss<0.000125][-1])

In [ ]:
%matplotlib inline  
import GPy
import GPyOpt
from numpy.random import seed

bounds = [{'name': 'latent_dim', 'type': 'discrete', 'domain':(3, 4, 5, 6, 7, 8)},
          {'name': 'outer_layer_width', 'type': 'discrete', 'domain':(16, 32, 64)},
          {'name': 'inner_layer_width', 'type': 'discrete', 'domain':(16, 32, 64)},
          {'name': 'beta', 'type': 'continuous', 'domain':(0,1)},
          {'name': 'batch_size', 'type': 'discrete', 'domain':(1024)}]

max_iter = 1000
myProblem = GPyOpt.methods.BayesianOptimization(main, domain=bounds)
myProblem.run_optimization(max_iter)

In [ ]:
# evaluate results of optimization
myProblem.plot_convergence()http://127.0.0.1:3244/?token=a81ca675177d942935a0d3476cf95d0923ca4264598f5f93
print('Writing evaluation results... ')
param1 = myProblem.get_evaluations()[0][:,0].flatten()
param2 = myProblem.get_evaluations()[0][:,1].flatten()
param3 = myProblem.get_evaluations()[0][:,2].flatten()
param4 = myProblem.get_evaluations()[0][:,3].flatten()
param5 = myProblem.get_evaluations()[0][:,4].flatten()
out = -1*myProblem.get_evaluations()[1][:].flatten()
opt_results = {'latent_dim': param1,
               'outer_layer_width': param2,
               'inner_layer_width': param3,
               'beta': param4,
               'batch_size': param5
               'efficiency': out}

evals = pd.DataFrame(opt_results)
print(evals)

print('The value of (latent_dim, outer_layer width, inner_layer_width, beta) that maximizes efficiency is: \n'+str(myProblem.x_opt))
print('The the max efficiency found is: '+str(-1*myProblem.fx_opt))